In [ ]:
import sys

In [ ]:
# Install a pip package in the current Jupyter kernel
!{sys.executable} -m pip install -e '../../../Wielder/'
!{sys.executable} -m pip install -e '../'

In [ ]:
# POC of how to install and uninstall local packages

# !{sys.executable} -m pip install -e '../../../pep-services/src/'
# !{sys.executable} -m pip uninstall wield-services -y

In [ ]:
from wielder.util.hocon_util import resolve_ordered

project_conf = '../conf/project.conf'

ordered_project_files = [project_conf]

conf = resolve_ordered(
    ordered_conf_paths=ordered_project_files)

print(conf)

In [ ]:
# Bee Hive data https://drive.google.com/file/d/142IBcs6OyQiJxO7owPfkEBFbkrudnh0g/view?usp=sharing

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [ ]:
# Create and start Spark session
spark = SparkSession.builder.appName("BeeHive").getOrCreate()

In [ ]:
#Create schema for the data

# All column names
headers = conf['population']['cols_name']

# All column names that their values are doubles
cols_double = conf['population']['cols_double']

# All column names that their values are integers
cols_integer = conf['population']['cols_integer']

def struct_field(header, doubles, integers):
    
    # Create a field for double type column
    if header in doubles:
        return StructField(header, DoubleType())
    
    # Create a field for integer type column
    if header in integers:
        return StructField(header, IntegerType())
    
    # # Create a field for string type column
    return StructField(header, StringType())

# Create all the fields
fields = [struct_field(header, cols_double, cols_integer) for header in headers]

# Create the schema from th e fields
schema = StructType(fields)

In [ ]:
# Read the data from the csv using the schema
df = spark.read.schema(schema).csv('BeeHiveTestData.csv')

df.show(5)

In [ ]:
# Remove all columns with the string remove in them
cols_to_keep = [x for x in df.columns if 'remove' not in x]
df = df.select(*cols_to_keep)

df.show(5)

In [ ]:
# Remove unnecessary columns
cols_to_drop = ('DaughtersEfficiencyScore', 'Father TYPE', 'X', 'Y', 'Z')
df_cleaned = df.drop(*cols_to_drop)

df_cleaned.show(5)

In [ ]:
# Count the number of bees for each Father SIZE
df_cleaned = df_cleaned.groupBy('Father SIZE').count()

df_cleaned.show()

In [ ]:
%matplotlib notebook
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Convert from pyspark data frame to numpy array
df_cleaned_np = np.array(df_cleaned.select('Father SIZE', 'count').collect())

df_cleaned_np

In [ ]:
# Create bar chart based on amount of bees in each Father SIZE group

# Create plot figure and axes
fig, ax = plt.subplots(figsize =(8, 7))

# Create bars and their labels
ax.bar([str(i) for i in df_cleaned_np[:,0]], df_cleaned_np[:,1])

# Add x, y gridlines
ax.grid( color ='grey', linestyle ='-.', linewidth = 0.5, alpha = 0.2)

for bar in ax.patches:
   
  # Using Matplotlib's annotate function and
  # passing the coordinates where the annotation shall be done
  # x-coordinate: bar.get_x() + bar.get_width() / 2
  # y-coordinate: bar.get_height()
  # free space to be left to make graph pleasing: (0, 8)
  # ha and va stand for the horizontal and vertical alignment
    ax.annotate(format(bar.get_height(), '.0f'),
                   (bar.get_x() + bar.get_width() / 2,
                    bar.get_height()), ha='center', va='center',
                   size=15, xytext=(0, 8),
                   textcoords='offset points')
    

# Add Plot Title
ax.set_title('Bee population distribution by father size', fontsize = 25)

# Set x axis name
ax.set_xlabel("Father size", fontsize =18)

# Set y axis name
ax.set_ylabel('Bees amout', fontsize =18)

# Show plot, not necessary but used to remove unwanted output
plt.show()

In [ ]:
# Create horizonatal bar chart based on amount of bees in each Father SIZE group

# Create plot figure and axes
fig, ax = plt.subplots(figsize =(9, 8))
 
# # Create horizontal bars and their labels
ax.barh([str(i) for i in df_cleaned_np[:,0]], df_cleaned_np[:,1])
 
# Add x, y gridlines
ax.grid( color ='grey', linestyle ='-.', linewidth = 0.5, alpha = 0.2)
 
# Add annotation to bars
for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5,
             str(round((i.get_width()), 2)),
             fontsize = 10, fontweight ='bold',
             color ='grey')
    
# Add Plot Title
ax.set_title('Bee population distribution by father size', fontsize = 25)

# Set x axis name
ax.set_xlabel('Bees amout', fontsize =18)

# Set y axis name
ax.set_ylabel('Father size', fontsize =18)

# Show plot, not necessary but used to remove unwanted output
plt.show()

In [ ]:
# Create pie chart based on amount of bees (in %) in each Father SIZE group

# Create plot figure and axes
fig1, ax1 = plt.subplots()

# Create pie slices (in %) and their labels
ax1.pie(df_cleaned_np[:,1], labels=df_cleaned_np[:,0], autopct='%1.2f%%')

# Equal aspect ratio ensures that pie is drawn as a circle.
ax1.axis('equal')  

# Show plot, not necessary but used to remove unwanted output
plt.show()